# Base-line Dense network

In [0]:
# Install TensorFlow
try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass

import tensorflow as tf
tf.__version__

'2.2.0'

In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
!apt install fluidsynth
!pip install midi2audio
!pip install pretty_midi

In [0]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization, Dense, Dropout, Input, Flatten
from tensorflow.keras.callbacks import ModelCheckpoint, LambdaCallback
from tensorflow.keras.utils import plot_model
import numpy as np
import pylab as plt
import glob

import sys
sys.path.insert(1, '/content/gdrive/My Drive/Music Gen/import/')
import models
import trainFac
import midiUtils

## NN Architecture


In [4]:
class Model_Dense_2Din_1Dout(models.Model_1Dout):

    def __init__(self, input_length):
        super().__init__(input_length)
        self.createModel()

    def createModel(self):
        model = Sequential()

        model.add(Input(shape=(self.input_length, 48, 1)))
        model.add(Flatten())

        # input: 120 * 48 = 5760
        model.add(Dense(1440, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        # output: 5760 / 4 = 1440

        # input: 1440
        model.add(Dense(360, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))
        # output: 1440 / 4 = 360

        # input: 360
        model.add(Dense(90, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(0.3))    
        # output: 360 / 4 = 90

        # input: 90
        model.add(Dense(48, activation='sigmoid'))
        self.model = model


modelCompound = Model_Dense_2Din_1Dout(120)

modelCompound.model.load_weights('/content/gdrive/My Drive/Music Gen/networks/dense_reference/train1/checkpoints/dense-20-accuracy0.4055.hdf5')
modelCompound.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 5760)              0         
_________________________________________________________________
dense (Dense)                (None, 1440)              8295840   
_________________________________________________________________
batch_normalization (BatchNo (None, 1440)              5760      
_________________________________________________________________
dropout (Dropout)            (None, 1440)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 360)               518760    
_________________________________________________________________
batch_normalization_1 (Batch (None, 360)               1440      
_________________________________________________________________
dropout_1 (Dropout)          (None, 360)               0

## Training

In [0]:
paths = trainFac.Paths(
    rootDir = '/content/gdrive/My Drive/Music Gen',
    trainPath = 'train_batches/train_7840_chorus_npz_500',
    testPath  = 'test_batches/test_800_chorus_npz_100',
    testSongPath = 'test_generation/test_songs',
    testGenerateSavePath = 'networks/dense_reference/train1/generate',
    checkpointBasePath = 'networks/dense_reference/train1/checkpoints',
    checkpointName='dense-{epoch:02d}-accuracy{val_accuracy:.4f}.hdf5')
# create paths if they dont exist
paths.createSavePaths()

trainFacility = trainFac.TrainFacility(modelCompound, paths)
trainFacility.compile()

In [0]:
history = trainFacility.fit(epochs=20)

Epoch 1/20
 9996/10000 [============================>.] - ETA: 0s - loss: 0.0958 - accuracy: 0.3982
Epoch 00001: saving model to /content/gdrive/My Drive/Music Gen/networks/dense_reference/train1/checkpoints/dense-01-accuracy0.4775.hdf5
create save path
 -- Generating songs after epoch: 0
 -- Generating songs finished
10000/10000 [==============================] - 174s 17ms/step - loss: 0.0958 - accuracy: 0.3982 - val_loss: 0.0380 - val_accuracy: 0.4775
Epoch 2/20
 9997/10000 [============================>.] - ETA: 0s - loss: 0.0690 - accuracy: 0.4498
Epoch 00002: saving model to /content/gdrive/My Drive/Music Gen/networks/dense_reference/train1/checkpoints/dense-02-accuracy0.4848.hdf5
10000/10000 [==============================] - 102s 10ms/step - loss: 0.0690 - accuracy: 0.4498 - val_loss: 0.0348 - val_accuracy: 0.4848
Epoch 3/20
 9996/10000 [============================>.] - ETA: 0s - loss: 0.0636 - accuracy: 0.4573
Epoch 00003: saving model to /content/gdrive/My Drive/Music Gen/net

In [0]:
import json
with open('/content/gdrive/My Drive/Music Gen/networks/dense_reference/train1/history.json', 'a') as fp:
    json.dump(history.history, fp, indent=4)

## Experiments

In [0]:
net_name = 'Dense'
save_path = '/content/gdrive/My Drive/Music Gen/results/dense/'

In [6]:
song_name = "Can't Stop"
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/cant_stop.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Hells Bells'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/hells_bells.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Layla'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/layla.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Nothing Else Matters'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/nothing_else_matters.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Smells Like Teen Spirit'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/smells_like_teen_spirit.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Under The Bridge'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/under_the_bridge.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Unforgiven 2'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/unforgiven_2.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Unforgiven'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/unforgiven.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.

In [0]:
song_name = 'Wonderwall'
seedPath = '/content/gdrive/My Drive/Music Gen/test_generation/test_songs/wonderwall.npz'

midi = modelCompound.generateMidi(
    seedPath,
    280,
    threshold=0.1)

midiUtils.plotMidi(save_path, net_name, song_name, midi)
midiUtils.playMidi(midi, save_path + midiUtils.joinNames(net_name, song_name))

Output hidden; open in https://colab.research.google.com to view.